In [ ]:

   def json2csv_depth(filename):
    """takes the json file COCO labeling format and 
    extracts annotations in a csv format, where each line consists of:
    Image id, object label, x_min, y_min, x_max, ymax
    The function return all image ids(all_ids) and all images with 
    annotations (all_ids_ann) 
    """
    import pandas as pd
    import json
    import os


    s = json.load(open(filename, 'r'))
#    print(json.dumps(s, indent=4, sort_keys=True)
    out_file = filename[:-5] + '.csv'
    
    out = open(out_file, 'w',encoding="utf-8")
    out.write('id,label,x1,y1,x2,y2\n')

    all_ids = []
    
    for im in s['images']:
        all_ids.append(im['id'])
        
        
    all_ids_ann = []            
    #iterating over annotations 
    for ann in s['annotations']:
        image_id = ann['image_id']
        label = ann['category_id']
        attrib = ann['attributes']
#        print(attrib)
        if 'name' in attrib:
            label = attrib['name']
        elif 'gender' in attrib:
            label = attrib['gender']
        elif 'Species ' in attrib:
            label = attrib['Species ']  
#            print(label)
            
#        if image_id<624:
#            print(image_id)
        for im in s['images']:
            if im['id'] == image_id:
                image_name = im['file_name']
                image_width= im['width']
                image_height= im['height']
                break


        for ca in s['categories']:
            if ca['id'] == label:
                label = ca['name']
                break          



        x1 = ann['bbox'][0]
        xmin=x1/image_width
        x2 = ann['bbox'][0] + ann['bbox'][2]
        xmax=x2/image_width
        y1 = ann['bbox'][1]
        ymin=y1/image_height
        y2 = ann['bbox'][1] + ann['bbox'][3]
        ymax=y2/image_height


        out.write('{},{},{},{},{},{}\n'.format(image_name,label.lower(), xmin, ymin, xmax, ymax))
        all_ids_ann.append(image_id)

    return [all_ids,all_ids_ann]

In [ ]:
filename ='/Users/seyran/Documents/GitHub/children_book_data/children_book_dataset/annotations/instances_default.json'
[all_ids,all_ids_ann]=json2csv_depth(filename)   

In [ ]:
all_ids = set(all_ids)
len(all_ids)
all_ids_ann = set(all_ids_ann)
len(all_ids_ann)
no_annotations = list(all_ids - all_ids_ann)
print(no_annotations)
len(no_annotations)

In [ ]:
# Remove images without any annotations
imagefolder ='/Users/seyran/Documents/GitHub/children_book_data/children_book_dataset/images_copy'
os.chdir(imagefolder)
#os.listdir(imagefolder)
s = json.load(open(filename, 'r'))
for image_id in no_annotations:
    for im in s['images']:
            if im['id'] == image_id:
                image_name = im['file_name']
                print(image_name)
    for root, dirs, files in os.walk(imagefolder):
        for filename in files:
            if filename == image_name:
                os.remove(filename)
                print(image_name)
                break

    
#    for image_id in no_annotations:
#        for im in s['images']:
#            if im['id'] == image_id:
#                image_name = im['file_name']
#                out.write('{},{},{},{},{},{}\n'.format(image_name, -1, -1, -1, -1, -1))
#    out.close()

    # Sort file by image id
    s1 = pd.read_csv(out_file)
    s1.sort_values('id', inplace=True)
    s1.to_csv(out_file, index=False)

In [ ]:
# merging two csv annotation files 
import pandas as pd
csv1="/Users/seyran/Documents/GitHub/children_book_data/children_book_dataset/annotations/instances_default.csv"
csv2="/Users/seyran/Documents/GitHub/children_book_data/children_book_dataset_incomplete/annotations/instances_default.csv"
a = pd.read_csv(csv1)
b = pd.read_csv(csv2)
out = pd.concat([a,b])
with open('/Users/seyran/Documents/GitHub/children_book_data/concatenated_annotations.csv', 'w', encoding='utf-8') as f:
    out.to_csv(f, index=False)